In [ ]:
import pandas as pd
import numpy as np
import re

In [ ]:
data = pd.read_csv('../data/census_data/2018_Data.csv')
data

In [ ]:
### get rid of specific columns
pattern1 = r'Margin of Error'
pattern2 = r'PERCENT ALLOCATED'
pattern3 = r'Unnamed:'

drop = []
    
for col in data.columns:
    if (re.search(pattern1,col) or re.search(pattern2,col) or re.search(pattern3,col)):
        drop.append(col)
#end
    
data = data.drop(columns = drop)


### clean up columns names
new_cols = []

for col in data.columns:
    new_col = col.replace('Estimate','').replace('Total', '').replace('!!',' ').strip()
    new_cols.append(new_col)
#end
    
data.columns = new_cols


### replace N and - with NaN
data = data.replace(to_replace='N',value=np.nan).replace(to_replace = '-', value = np.nan).replace('**',np.nan)


data

In [ ]:
pattern = r'over.AGE'
keep = ['Geographic Area Name', 'Workers 16 years and over', 'Workers 16 years and over who did not work at home',
        'Car, truck, or van -- drove alone Workers 16 years and over', 'Car, truck, or van -- carpooled Workers 16 years and over',
        'Public transportation (excluding taxicab) Workers 16 years and over']

for col in data.columns:
    if re.search(pattern,col):
        keep.append(col)
#end

In [ ]:
age = data[keep]
#age

In [ ]:
age = age.dropna(thresh = 24)
age = age.set_index('Geographic Area Name')
#age

In [ ]:
age.dtypes

In [ ]:
age = age.apply(pd.to_numeric)
age.dtypes

In [ ]:
drop = []
for ind, values in age.iterrows():
    if pd.isna(values['Workers 16 years and over who did not work at home']):
        drop.append(ind)
#end

In [ ]:
age = age.drop(index = drop)
age

In [ ]:
list(age.columns)

In [ ]:
for row in age.itertuples():
    comm_total  = row[2]
    drove_alone = row[3]
    carpool     = row[4]
    pub_transit = row[5]

    for i in range(6,34):
        if i <= 11: # total
            try:
                new_val = int(comm_total*(row[i]/100))
                age.loc[row.Index, age.columns[i-1]] = new_val
            except:
                age.loc[row.Index, age.columns[i-1]] = np.nan
            #end
        elif ((i >= 13) and (i <= 18)): # drove alone
            try:
                new_val = int(drove_alone*(row[i]/100))
                age.loc[row.Index, age.columns[i-1]] = new_val
            except:
                age.loc[row.Index, age.columns[i-1]] = np.nan
                #end
        elif ((i >= 20) and (i <= 25)): # carpool
            try:
                new_val = int(carpool*(row[i]/100))
                age.loc[row.Index, age.columns[i-1]] = new_val
            except:
                age.loc[row.Index, age.columns[i-1]] = np.nan
            #end
        elif ((i >= 27) and (i <= 32)): # public transit
            try:
                new_val = int(pub_transit*(row[i]/100))
                age.loc[row.Index, age.columns[i-1]] = new_val
            except:
                age.loc[row.Index, age.columns[i-1]] = np.nan
            #end

In [ ]:
age

In [ ]:
def filter_data(path, category = 'AGE'):
    ### read in data
    data = pd.read_csv(path)
    
    
    ### create name of file for saving
    year = int(path.split('_data/')[1].split('_Data')[0])
    file = str(year) + f'_Data_{str.capitalize(category)}'
    
    ### get rid of specific columns
    #print('Get rid of specific columns')
    pattern1 = r'Margin of Error'
    pattern2 = r'PERCENT ALLOCATED'
    pattern3 = r'Unnamed:'

    drop = []
    
    for col in data.columns:
        if (re.search(pattern1,col) or re.search(pattern2,col) or re.search(pattern3,col)):
            drop.append(col)
    #end
    
    data = data.drop(columns = drop)
    
    ### clean up columns names
    #print('Clean up columns names')
    new_cols = []

    for col in data.columns:
        new_col = col.replace('Estimate','').replace('Total', '').replace('!!',' ').replace('  ',' ').strip()
        new_cols.append(new_col)
    #end
    
    data.columns = new_cols
    
    
    ### replace N and - with NaN
    #print('Replace N and - with NaN')
    data = data.replace(to_replace='N',value=np.nan).replace(to_replace = '-', value = np.nan).replace('**',np.nan)
    
    
    ### remove rows that are mostly null or where 'Workers 16 years and over who did not work from home' is null
    ### ALSO CHECK IF THIS COLUMN EXISTS!
    #print('Determine j')
    data = data.dropna(thresh = 25)
    
    if 'Workers 16 years and over who did not work from home' in data.columns:
        j = 1 # this indicates that my original code will work with this data
        print('j is',j)
        
        drop = []
        
        for ind, values in data.iterrows():
            if pd.isna(values['Workers 16 years and over who did not work from home']):
                drop.append(ind)
        #end

    elif year == 2018:
        j = 2
        print('j is',j)
        drop = []
        
        for ind, values in data.iterrows():
            if pd.isna(values['Workers 16 years and over who did not work at home']):
                drop.append(ind)
        #end
    
    else:
        j = 0
        print('j is',j)
        drop = []
        
        for ind, values in data.iterrows():
            if pd.isna(values['Workers 16 years and over who did not work at home']):
                drop.append(ind)
        #end
    
    data = data.drop(index = drop)
    
    
    ### keep specific commute related columns
    #print('Keep specific commute related columns')

    if j%2 == 0: # that is, j = 0 or j = 2
        pattern1 = r'AGE.\d'
        pattern2 = r'Median age'
    else:
        pattern1 = r'over.AGE'
        pattern2 = r'PLACEHOLDER TEXT' # There wasn't any need for a second pattern but I needed the code to be consistent
    
    if j%2 == 0:
        keep = ['Geographic Area Name','Workers 16 years and over','Workers 16 years and over who did not work at home',
                'Car, truck, or van -- drove alone Workers 16 years and over', 'Car, truck, or van -- carpooled Workers 16 years and over',
                'Public transportation (excluding taxicab) Workers 16 years and over']
    else:
        keep = ['Geographic Area Name','Workers 16 years and over','Workers 16 years and over who did not work from home',
                'Car, truck, or van -- drove alone Workers 16 years and over','Car, truck, or van -- carpooled Workers 16 years and over',
                'Public transportation (excluding taxicab) Workers 16 years and over']
        
    for col in data.columns:
        if re.search(pattern1,col) or re.search(pattern2,col):
            keep.append(col)
    #end
    
    data = data[keep]

    
    ### Set county as index
    data = data.set_index('Geographic Area Name')
    
    
    ### make dtypes numeric
    data = data.apply(pd.to_numeric)
    


    ### get rid of percentages by translating them to integers BASED ON j
    if j == 0:
        for row in data.itertuples():
            comm_total  = row[2]
            drove_alone = row[3]
            carpool     = row[4]
            pub_transit = row[5]

            for i in range(6,30):
                if i%4 == 2: # total
                    try:
                        new_val = int(comm_total*(row[i]/100))
                        data.loc[row.Index, data.columns[i-1]] = new_val
                    except:
                        data.loc[row.Index, data.columns[i-1]] = np.nan
                    #end
                elif i%4 == 3: # drove alone
                    try:
                        new_val = int(drove_alone*(row[i]/100))
                        data.loc[row.Index, data.columns[i-1]] = new_val
                    except:
                        data.loc[row.Index, data.columns[i-1]] = np.nan
                        #end
                elif i%4 == 0: # carpool
                    try:
                        new_val = int(carpool*(row[i]/100))
                        data.loc[row.Index, data.columns[i-1]] = new_val
                    except:
                        data.loc[row.Index, data.columns[i-1]] = np.nan
                    #end
                elif i%4 == 1: # public transit
                    try:
                        new_val = int(pub_transit*(row[i]/100))
                        data.loc[row.Index, data.columns[i-1]] = new_val
                    except:
                        data.loc[row.Index, data.columns[i-1]] = np.nan
                    #end
                    
    elif j == 1:
        for row in data.itertuples():
            comm_total  = row[2]
            drove_alone = row[3]
            carpool     = row[4]
            pub_transit = row[5]
            
            for i in range(6,33):                
                if (i <= 11): # total
                    try:
                        new_val = int(comm_total*(row[i]/100))
                        data.loc[row.Index, data.columns[i-1]] = new_val
                    except:
                        data.loc[row.Index, data.columns[i-1]] = np.nan
                    #end
                    
                elif ((i >= 13) and (i <= 18)): # drove alone
                    try:
                        new_val = int(drove_alone*(row[i]/100))
                        data.loc[row.Index, data.columns[i-1]] = new_val
                    except:
                        data.loc[row.Index, data.columns[i-1]] = np.nan
                        #end
                        
                elif ((i >= 20) and (i <= 25)): # carpool
                    try:
                        new_val = int(carpool*(row[i]/100))
                        data.loc[row.Index, data.columns[i-1]] = new_val
                    except:
                        data.loc[row.Index, data.columns[i-1]] = np.nan
                    #end
                    
                elif ((i >= 27) and (i <= 32)): # public transit
                    try:
                        new_val = int(pub_transit*(row[i]/100))
                        data.loc[row.Index, data.columns[i-1]] = new_val

                    except:
                        data.loc[row.Index, data.columns[i-1]] = np.nan
                    #end
    else:
        for row in data.itertuples():
            comm_total  = row[2]
            drove_alone = row[3]
            carpool     = row[4]
            pub_transit = row[5]
            
            for i in range(6,31):                
                if (i < 12): # total
                    try:
                        new_val = int(comm_total*(row[i]/100))
                        data.loc[row.Index, data.columns[i-1]] = new_val
                    except:
                        data.loc[row.Index, data.columns[i-1]] = np.nan
                    #end
                    
                elif i == 12:
                    continue
                    
                elif i%3 == 1: # drove alone
                    try:
                        new_val = int(drove_alone*(row[i]/100))
                        data.loc[row.Index, data.columns[i-1]] = new_val
                    except:
                        data.loc[row.Index, data.columns[i-1]] = np.nan
                        #end
                        
                elif i%3 == 2: # carpool
                    try:
                        new_val = int(carpool*(row[i]/100))
                        data.loc[row.Index, data.columns[i-1]] = new_val
                    except:
                        data.loc[row.Index, data.columns[i-1]] = np.nan
                    #end
                    
                elif i%3 == 0: # public transit
                    try:
                        new_val = int(pub_transit*(row[i]/100))
                        data.loc[row.Index, data.columns[i-1]] = new_val

                    except:
                        data.loc[row.Index, data.columns[i-1]] = np.nan
                    #end
        #end
    
    
    ### Clean up column names some more
    pattern0 = r'Workers 16 years and over who did not work at home'
    pattern1 = r'Workers 16 years and over who did not work from home'
    pattern2 = r'Estimate'
    pattern3 = r'(excluding taxicab)'

    new_cols = {}

    for col in data.columns:
        if (col != 'Workers 16 years and over who did not work from home' 
            and col != 'Workers 16 years and over who did not work from home') and (re.search(pattern1,col) 
                                                                                 or re.search(pattern2,col) 
                                                                                 or re.search(pattern3,col)):
            new_col = col.replace(str(pattern1),'').replace(str(pattern2),'').replace(str(pattern3),'').strip()
            new_cols[col] = new_col
    #end
    
    data = data.rename(columns = new_cols)
    data = data.dropna(thresh = 25)
    
    ### save county file
    data.to_csv(f'../data/cleaned_data/{file}_County.csv')
    
    
    ### set up aggregations for group by
    aggs = {}
    
    for col in data.columns:        
        num = data.columns.get_loc(col)
        
        if j == 0:
            if num < 29:
                aggs[col] = np.sum
            else:
                aggs[col] = np.median
            #end
        elif j == 1:
            if num in [11,18,25,32]:
                aggs[col] = np.median
            else:
                aggs[col] = np.sum
            #end
        elif j == 2:
            if num in [11,30,31,32]:
                aggs[col] = np.median
            else:
                aggs[col] = np.sum
    #end
    
    
    ### add State column (so we can group by it)
    states = []

    for ind, values in data.iterrows():
        state = ind.split(', ')[1]
        states.append(state)
    #end
    
    data['State'] = states
    
    
    ### finally, group by state and return df
    
    data = data.groupby('State').agg(aggs).replace(np.nan, 0).astype(int)#.replace(to_replace=0,value=np.nan)
    
    data.to_csv(f'../data/cleaned_data/{file}_State.csv')
    
    return data

In [ ]:
df = filter_data('../data/census_data/2021_Data.csv')
df.loc[df['Public transportation  Workers 16 years and over'] > 5000]

In [ ]:
df.columns#['Public transportation Workers 16 years and over']

In [ ]:
df.loc['Houston County, Alabama', 'Workers 16 years and over AGE 16 to 19 years']

In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
data = pd.read_csv('../data/census_data/2010_Data.csv')

### get rid of specific columns
pattern1 = r'Margin of Error'
pattern2 = r'PERCENT ALLOCATED'
pattern3 = r'Unnamed:'

drop = []
    
for col in data.columns:
    if (re.search(pattern1,col) or re.search(pattern2,col) or re.search(pattern3,col)):
        drop.append(col)
#end
    
data = data.drop(columns = drop)

### clean up columns names
new_cols = []

for col in data.columns:
    new_col = col.replace('Estimate','').replace('Total', '').replace('!!',' ').replace('  ',' ').strip()
    new_cols.append(new_col)
#end
    
data.columns = new_cols

list(data.columns)

In [ ]:
pattern1 = r'AGE.\d'
pattern2 = r'Median age'

keep = ['Geographic Area Name','Workers 16 years and over','Car, truck, or van -- drove alone Workers 16 years and over',
        'Car, truck, or van -- carpooled Workers 16 years and over','Public transportation (excluding taxicab) Workers 16 years and over']

for col in data.columns:
    if re.search(pattern,col) or re.search(pattern2,col):
        print('KEEPING:',col)
        keep.append(col)

In [ ]:
data = data[keep]
data = data.replace('N',np.nan)
data = data.dropna(thresh = 20)
data = data.set_index('Geographic Area Name')
data

In [ ]:
df = filter_data('../data/census_data/2021_Data.csv')
df

Now let's get everything for each table!

In [ ]:
for i in range (2010,2023):
    if i != 2020:
        print(i)
        
        try:
            path = f'../data/census_data/{i}_Data.csv'
            new_df = filter_data(path)
            #new_df.to_csv(f'../data/cleaned_data/{i}_Data_State.csv')
            print('Completed')
        except:
            print('Skipped')
            continue
#end